In [ ]:
import tensorflow as tf
# GPU memory growth configuration
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

**Dataset details**

The dataset contains two classes - REAL and FAKE.

For REAL, we collected the images from Krizhevsky & Hinton's CIFAR-10 dataset

For the FAKE images, we generated the equivalent of CIFAR-10 with Stable Diffusion version 1.4

There are 100,000 images for training (50k per class) and 20,000 for testing (10k per class)

# Load the dataset

In [ ]:
import numpy as np
import os
import cv2
from keras.utils import to_categorical
# Define paths to the train and test directories
train_dir_real = '/kaggle/input/cifake-real-and-ai-generated-synthetic-images/train/REAL'
train_dir_fake = '/kaggle/input/cifake-real-and-ai-generated-synthetic-images/train/FAKE'
test_dir_real = '/kaggle/input/cifake-real-and-ai-generated-synthetic-images/test/REAL'
test_dir_fake = '/kaggle/input/cifake-real-and-ai-generated-synthetic-images/test/FAKE'

# Function to load images and labels
def load_images_and_labels(directory, label):
    images = []
    labels = []
    for filename in os.listdir(directory):
        img_path = os.path.join(directory, filename)
        img = cv2.imread(img_path)
        img = cv2.resize(img, (32, 32)) 
        images.append(img)
        labels.append(label)
    return images, labels

# Load REAL train images and labels
real_train_images, real_train_labels = load_images_and_labels(train_dir_real, label=0)

# Load FAKE train images and labels
fake_train_images, fake_train_labels = load_images_and_labels(train_dir_fake, label=1)

# Load REAL test images and labels
real_test_images, real_test_labels = load_images_and_labels(test_dir_real, label=0)

# Load FAKE test images and labels
fake_test_images, fake_test_labels = load_images_and_labels(test_dir_fake, label=1)

# Concatenate REAL and FAKE train/test data and labels
X_train = np.concatenate((real_train_images, fake_train_images), axis=0)
y_train = np.concatenate((real_train_labels, fake_train_labels), axis=0)
X_test = np.concatenate((real_test_images, fake_test_images), axis=0)
y_test = np.concatenate((real_test_labels, fake_test_labels), axis=0)

# Convert labels to one-hot encoding
y_train = np.eye(2)[y_train]
y_test = np.eye(2)[y_test]

# Normalize image data
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# 1. test for the data to work correctly

In [ ]:
print("Total number of REAL images in y_train:", np.sum(y_train[:, 0]))
print("Total number of FAKE images in y_train:", np.sum(y_train[:, 1]))

print("Number of REAL images in the first 50000 samples:", np.sum(y_train[:50000, 0]))
print("Number of FAKE images in the first 50000 samples:", np.sum(y_train[:50000, 1]))

print("Number of REAL images in the second 50000 samples:", np.sum(y_train[50000:, 0]))
print("Number of FAKE images in the second 50000 samples:", np.sum(y_train[50000:, 1]))

In [ ]:
print("Total number of REAL images in y_test:", np.sum(y_test[:, 0]))
print("Total number of FAKE images in y_test:", np.sum(y_test[:, 1]))

print("Number of REAL images in the first 10000 samples:", np.sum(y_test[:10000, 0]))
print("Number of FAKE images in the first 10000 samples:", np.sum(y_test[:10000, 1]))

print("Number of REAL images in the second 10000 samples:", np.sum(y_test[10000:, 0]))
print("Number of FAKE images in the second 10000 samples:", np.sum(y_test[10000:, 1]))

*** Guaranteeing balance between real and fake images in each batch for training an image classification model**

In [ ]:
# Shuffle the order of REAL and FAKE train data
shuffled_indices_train = np.arange(X_train.shape[0])
np.random.shuffle(shuffled_indices_train)

# Use the same set of shuffled indices for both images and labels
X_train = X_train[shuffled_indices_train]
y_train = y_train[shuffled_indices_train]

# Shuffle the order of REAL and FAKE test data
shuffled_indices_test = np.arange(X_test.shape[0])
np.random.shuffle(shuffled_indices_test)

# Use the same set of shuffled indices for both images and labels
X_test = X_test[shuffled_indices_test]
y_test = y_test[shuffled_indices_test]

# 2. test for the data to work correctly

In [ ]:
# Print the class distribution in y_train
print("Class distribution in y_train:", np.sum(y_train, axis=0))

# Print the class distribution in y_test
print("Class distribution in y_test:", np.sum(y_test, axis=0))


In [ ]:
print("Total number of REAL images in y_test:", np.sum(y_test[:, 0]))
print("Total number of FAKE images in y_test:", np.sum(y_test[:, 1]))

print("Number of REAL images in the first 10000 samples:", np.sum(y_test[:10000, 0]))
print("Number of FAKE images in the first 10000 samples:", np.sum(y_test[:10000, 1]))

print("Number of REAL images in the second 10000 samples:", np.sum(y_test[10000:, 0]))
print("Number of FAKE images in the second 10000 samples:", np.sum(y_test[10000:, 1]))

In [ ]:
print("Total number of REAL images in y_train:", np.sum(y_train[:, 0]))
print("Total number of FAKE images in y_train:", np.sum(y_train[:, 1]))

print("Number of REAL images in the first 50000 samples:", np.sum(y_train[:50000, 0]))
print("Number of FAKE images in the first 50000 samples:", np.sum(y_train[:50000, 1]))

print("Number of REAL images in the second 50000 samples:", np.sum(y_train[50000:, 0]))
print("Number of FAKE images in the second 50000 samples:", np.sum(y_train[50000:, 1]))


# Build the CNN model

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same', input_shape=(32, 32, 3), activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Flatten())

model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(2, activation='softmax'))

In [ ]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=15, batch_size=100)

In [ ]:
import pandas as pd
pd.DataFrame(history.history).plot()

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Predictions on test set
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# Calculate confusion matrix
cm = confusion_matrix(y_true_classes, y_pred_classes, labels=[0, 1])

# Display confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["REAL", "FAKE"])
disp.plot(cmap='viridis', values_format='d')
plt.show()


In [ ]:
from sklearn.metrics import classification_report

# Classification report
print(classification_report(y_true_classes, y_pred_classes, target_names=["REAL", "FAKE"]))


In [ ]:
labels = ["REAL", "FAKE"]

fig, axes = plt.subplots(ncols=7, nrows=3, sharex=False,
    sharey=True, figsize=(17, 8))
index = 0
for i in range(3):
    for j in range(7):
        actual_label = labels[np.argmax(y_test[index])]
        predicted_label = labels[np.argmax(predictions[index])]
        axes[i, j].set_title(f'actual: {actual_label}\npredicted: {predicted_label}')
        axes[i, j].imshow(X_test[index], cmap='gray')
        axes[i, j].get_xaxis().set_visible(False)
        axes[i, j].get_yaxis().set_visible(False)
        index += 1

# Add debug information
print("Sample Order:")
print(np.argmax(y_test[20:50], axis=1))
print("Predictions Order:")
print(np.argmax(predictions[20:50], axis=1))

plt.show()


In [ ]:
predictions = model.predict(X_test)
for i in range(10):
    actual_label = labels[np.argmax(y_test[i])]
    predicted_label = labels[np.argmax(predictions[i])]
    print(f'Sample {i + 1} - Actual: {actual_label}, Predicted: {predicted_label}')